In [10]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
from bson.json_util import dumps, loads
import bson
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
#DEFINE USERNAME AND PASSWORD

username = "aacuser"
password = "Vi3w3d2021"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.readAll({}))


#########################
# Dashboard Layout / View
#########################
                          
app = JupyterDash('SimpleExample')

#Import Image and encode
image_filename = 'GSlogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#MAIN LAYOUT
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #insert image and tag line
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H2('www.snhu.edu'))),
    html.Br(),
    #insert unique identifier
    html.Center(html.H2(children='Chris Gollnick SNHU CS-340 Dashboard')),
    html.Hr(),
    #interactive filter buttons
    dcc.RadioItems(
        id='rd',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster Rescue ', 'value': 'Disaster Rescue or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Water Rescue',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    #dash frame and datatable
    html.Div(id='dash-frame', children = [
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                 #Set up the features for your interactive data table to make it user-friendly for your client
                editable = False,
                filter_action="native",
                sort_action="native",
                sort_mode="multi",
                column_selectable=False,
                row_selectable=False,
                row_deletable=False,
                selected_columns=[],
                selected_rows=[],
                page_action="native",
                page_current=0,
                page_size=10,
            )]),
    
    html.Hr(),
    html.Br(),
    #setup for graph and map on same row
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    
 
    ])


#############################################
# Interaction Between Components / Controller
#############################################
#filter button callbacks
@app.callback(
    dash.dependencies.Output('dash-frame', 'children'),
    [dash.dependencies.Input('rd', 'value')]
    )
def update_output(value):
    df = pd.DataFrame.from_records(shelter.readAll({}))
    
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"Labrador Retriever Mix"},
                                                        {"breed":"Chesapeake Bay Retriever"},
                                                        {"breed":"Newfoundland"}
                                                       ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                    "sex_upon_outcome":"Intact Female"}))
    if value == 'Mountain or Wilderness Rescue':
    
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                        {"breed":"Alaskan Malamute"},
                                                        {"breed":"Old English Sheepdog"},
                                                        {"breed":"Siberian Husky"} ,
                                                        {"breed":"Rottweiler"}  
                                                       ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                    "sex_upon_outcome":"Intact Male"}))
    if value == 'Disaster Rescue or Individual Tracking':
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                        {"breed":"Doberman Pinscher"},
                                                        {"breed":"Golden Retriever"},
                                                        {"breed":"Bloodhound"} ,
                                                        {"breed":"Rottweiler"}  
                                                       ],"age_upon_outcome_in_weeks":{'$gt':20,'$lt':300},
                                                    "sex_upon_outcome":"Intact Male"}))
        
    return [
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                editable=True,
                style_cell={'textAlign': 'left'},
                 style_cell_conditional=[
                {
                    'if': {'column_id': s},
                    'textAlign': 'left'
                } for s in ['animal_type','color','breed','animal_id']
            ],
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(0, 128, 0)'
                }
            ],
            style_header={
                'backgroundColor': 'rgb(0, 238, 0)',
                'fontWeight': 'bold'
            },
            
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="multi",
            row_deletable=True,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,

            )
        
       ]
#select column on datatable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
#graph callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #Add Graph
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Percent Breeds In Shelter')
        )    
    ]                         
                       
#map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,10]),
                    html.H3("Animal Color"),
                    html.P(dff.iloc[0,5]),
                    html.H3("Animal ID"),
                    html.P(dff.iloc[0,2]),
                    
                ])
            ])
        ])
    ]




app
